# Config

> Configuration constants, directory management, and base application schema

In [ ]:
#| default_exp core.config

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from pathlib import Path
from typing import Dict, Any, Optional, List

In [ ]:
#| export
# Default configuration storage directory
DEFAULT_CONFIG_DIR = Path("configs")

In [ ]:
#| export
def get_app_config_schema(
    app_title: str = "FastHTML Application",  # Default application title
    config_dir: str = "configs",  # Default configuration directory
    server_port: int = 5000,  # Default server port
    themes_enum: Optional[List[str]] = None,  # Optional list of theme values
    themes_enum_names: Optional[List[str]] = None,  # Optional list of theme display names
    default_theme: Optional[str] = None,  # Default theme value
    include_theme: bool = True,  # Whether to include theme selection
    **extra_properties  # Additional custom properties to add to the schema
) -> Dict[str, Any]:  # JSON Schema for application configuration
    """Generate a customizable application configuration schema.
    
    This function creates a JSON Schema for application settings that can be customized
    with your own defaults and additional properties.
    
    Returns:
        A JSON Schema dictionary with application configuration structure
    """
    
    schema = {
        "type": "object",
        "name": "general",
        "title": "Application Configuration",
        "menu_title": "Application",
        "description": "General application settings and configuration",
        "properties": {
            "app_title": {
                "type": "string",
                "title": "Application Title",
                "description": "Title displayed in the navbar and browser tab",
                "default": app_title,
                "minLength": 1,
                "maxLength": 100
            },
            "config_dir": {
                "type": "string",
                "title": "Configuration Directory",
                "description": "Directory where configuration files are stored",
                "default": config_dir,
                "pattern": '^[^<>:"|?*]+$'
            },
            "auto_open_browser": {
                "type": "boolean",
                "title": "Auto Open Browser",
                "description": "Automatically open browser when starting the application",
                "default": True
            },
            "server_port": {
                "type": "integer",
                "title": "Server Port",
                "description": "Port number for the web server",
                "default": server_port,
                "minimum": 1024,
                "maximum": 65535
            },
            "server_host": {
                "type": "string",
                "title": "Server Host",
                "description": "Host address for the web server",
                "default": "0.0.0.0",
                "enum": ["0.0.0.0", "127.0.0.1", "localhost"],
                "enumNames": ["All interfaces (0.0.0.0)", "Local only (127.0.0.1)", "Local only (localhost)"]
            },
            "debug_mode": {
                "type": "boolean",
                "title": "Debug Mode",
                "description": "Enable debug mode for development",
                "default": False
            },
            "reload_on_change": {
                "type": "boolean",
                "title": "Auto Reload",
                "description": "Automatically reload the application when files change (development only)",
                "default": False
            },
            "max_upload_size_mb": {
                "type": "number",
                "title": "Maximum Upload Size (MB)",
                "description": "Maximum file upload size in megabytes",
                "default": 100,
                "minimum": 1,
                "maximum": 10240
            },
            "session_timeout_minutes": {
                "type": "integer",
                "title": "Session Timeout (minutes)",
                "description": "Session timeout in minutes (0 for no timeout)",
                "default": 60,
                "minimum": 0,
                "maximum": 1440
            }
        },
        "required": ["app_title", "config_dir"]
    }
    
    # Add theme property if requested
    if include_theme and themes_enum and themes_enum_names:
        schema["onclick_save"] = """
        // Get the theme value from the form
        const themeSelect = document.querySelector('select[name="theme"]');
        if (themeSelect) {
            const theme = themeSelect.value;
            // Apply the theme immediately
            document.documentElement.setAttribute('data-theme', theme);
            // Persist the theme to localStorage
            localStorage.setItem('daisyui-theme', theme);
        }
    """
        
        schema["properties"]["theme"] = {
            "type": "string",
            "title": "Default Theme",
            "description": "Default application theme",
            "default": default_theme or themes_enum[0],
            "enum": themes_enum,
            "enumNames": themes_enum_names
        }
    
    # Add any extra custom properties
    schema["properties"].update(extra_properties)
    
    return schema

In [ ]:
# Example: Create a basic app config schema
basic_schema = get_app_config_schema(
    app_title="My FastHTML App",
    server_port=5001,
    include_theme=False
)

print(f"Schema name: {basic_schema['name']}")
print(f"Schema title: {basic_schema['title']}")
print(f"Default app_title: {basic_schema['properties']['app_title']['default']}")
print(f"Default server_port: {basic_schema['properties']['server_port']['default']}")
print(f"Has theme property: {'theme' in basic_schema['properties']}")

Schema name: general
Schema title: Application Configuration
Default app_title: My FastHTML App
Default server_port: 5001
Has theme property: False


In [ ]:
# Example: Create schema with DaisyUI themes
try:
    from cjm_fasthtml_daisyui.core.themes import DaisyUITheme
    
    theme_schema = get_app_config_schema(
        app_title="My Themed App",
        themes_enum=[theme.value for theme in DaisyUITheme],
        themes_enum_names=[theme.value.title() for theme in DaisyUITheme],
        default_theme=DaisyUITheme.LIGHT.value,
        include_theme=True
    )
    
    print(f"Has theme property: {'theme' in theme_schema['properties']}")
    print(f"Default theme: {theme_schema['properties']['theme']['default']}")
    print(f"Has onclick_save: {'onclick_save' in theme_schema}")
except ImportError:
    print("cjm_fasthtml_daisyui not available - skipping theme example")

Has theme property: True
Default theme: light
Has onclick_save: True


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()